### Assignment: Evolutionary Algorithm for Timetable Scheduling

#### Objective
Your task is to develop an **Evolutionary Algorithm (EA)** that optimizes a timetable for a set of courses based on predefined constraints. The algorithm should generate a valid schedule that meets the specified requirements while optimizing for minimal conflicts and balanced distribution of classes.

#### Problem Description
You are given a file (`classes.json`) containing a set of courses, their class types, scheduled days, and start times. Your goal is to create a feasible timetable that schedules all required classes while adhering to the given constraints.

#### Constraints
1. **Total Classes**: The timetable must contain **exactly 11 classes**.
2. **Course Distribution**:
   - **Tópicos de Física Moderna (TFM)**: 3 classes (**2 T1** and **1 TP**)
   - **Princípios de Programação Procedimental (PPP)**: 2 classes (**2 TP**)
   - **Comunicação Técnica (CT)**: 2 classes (**1 T1** and **1 PL**)
   - **Estatística**: 2 classes (**2 TP**)
   - **Análise Matemática II (AMII)**: 2 classes (**2 TP**)
3. **No Overlaps**: Two classes cannot be scheduled at the same time slot.
4. **Valid Time Slots**: Each class can only be scheduled in one of the available time slots provided in `classes.json`.

#### Input Format
The input file `classes.json` contains an array of objects, where each object has the following attributes:
- **Course**: The name of the course.
- **Class**: The type and section of the class (e.g., T1, TP1, PL1).
- **Day**: The scheduled day of the week.
- **Start Time**: The starting time of the class.

Example JSON entry:
```json
{
    "Course": "Análise Matemática II",
    "Class": "TP1",
    "Day": "Tuesday",
    "Start Time": "09:00"
}
```

#### Evolutionary Algorithm Requirements
Your **Evolutionary Algorithm** should follow these key principles:
1. **Representation**: Design an appropriate encoding for the timetable.
2. **Fitness Function**: Evaluate solutions based on:
   - Validity (whether all constraints are met)
   - Minimized conflicts
   - Balanced distribution of classes across time slots
3. **Parent Selection**: Implement a method for selecting promising solutions. Start by implementing the tournament selection.
4. **Crossover**: Define a mechanism to combine two parent solutions to create new timetables.
5. **Mutation**: Implement a mutation strategy to introduce diversity.
6. **Termination Condition**: Decide when the algorithm should stop (e.g., after a fixed number of generations or when there is no significant improvement).

After programming all of this you should implement the elitism mechanism.


Good luck, and happy coding!


In [80]:
# import libraries and load data from json
import random
import json
from datetime import datetime
import copy

NUMBER_OF_CLASSES_PER_WEEK = 11

# Load class data from JSON file
with open('classes.json', 'r', encoding='utf-8') as f:
    class_data = json.load(f)

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
times = ["09:00", "11:00", "14:00", "16:00", "18:00"]
courses_classe = {}
for i in class_data:
    courses_classe.setdefault(i['Course'], set()).add(i['Class'])

total_courses = len(courses_classe)
max_classes = max(len(classes) for classes in courses_classe.values())
total_days = len(days)
total_time_slots = len(times)

In [81]:
# check the contents:

## to check the given options for timetable selection
for c in class_data:
    print(c)

print("....")
## to access courses and classes
for cor in courses_classe:
    print(cor, " -> ", courses_classe[cor])

print(len(class_data))

{'Course': 'Análise Matemática II', 'Class': 'TP1', 'Day': 'Tuesday', 'Start Time': '09:00'}
{'Course': 'Análise Matemática II', 'Class': 'TP2', 'Day': 'Tuesday', 'Start Time': '09:00'}
{'Course': 'Análise Matemática II', 'Class': 'TP3', 'Day': 'Tuesday', 'Start Time': '09:00'}
{'Course': 'Estatística', 'Class': 'TP3', 'Day': 'Tuesday', 'Start Time': '09:00'}
{'Course': 'Comunicação Técnica', 'Class': 'T1', 'Day': 'Wednesday', 'Start Time': '09:00'}
{'Course': 'Análise Matemática II', 'Class': 'TP1', 'Day': 'Thursday', 'Start Time': '09:00'}
{'Course': 'Análise Matemática II', 'Class': 'TP2', 'Day': 'Thursday', 'Start Time': '09:00'}
{'Course': 'Análise Matemática II', 'Class': 'TP3', 'Day': 'Thursday', 'Start Time': '09:00'}
{'Course': 'Estatística', 'Class': 'TP3', 'Day': 'Thursday', 'Start Time': '09:00'}
{'Course': 'Tópicos de Física Moderna', 'Class': 'T1', 'Day': 'Monday', 'Start Time': '09:30'}
{'Course': 'Tópicos de Física Moderna', 'Class': 'T1', 'Day': 'Friday', 'Start Time':

# Initial Solution
Generates a random a chromosome that will represent a timetable

In [82]:
def generate_chromosome():
    # TODO YOUR CODE HERE
    # Generate a random chromosome that will represent a timetable
    # Create a binary chromosome
    # Initialize RANDOMLY WITH 0s and 1s
    chromosome = [random.choice([0, 1]) for _ in range(len(class_data))]
    return chromosome


teste = generate_chromosome()
print(teste)
print(len(teste))

# TFM - 2 T, 1 TP
# PPP - 2 TP
# CT - 1 T, 1 PL
# Estatistica - 2 TP
# AMII - 2 TP
# 2+1+2+1+1+2+2 = 11 classes per week

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]
50


In [83]:
def chromosome_to_timetable(chromosome): # generate the timetable object from the representation
    ''' from the genes you should generate the a list with elements that
    follow a structure similar to classes.json as follows:
        {
            "Course": course,
            "Class": classe,
            "Day": day,
            "Start Time": time
        }'''
    # TODO
    timetable = []
    for idx, bit in enumerate(chromosome):
        if bit == 1:
            timetable.append(class_data[idx])  # Add the selected class to the timetable
    return timetable

teste2 = chromosome_to_timetable(teste)
# make a better print
for i in teste2:
    print(i)

print(len(teste2))

{'Course': 'Análise Matemática II', 'Class': 'TP1', 'Day': 'Thursday', 'Start Time': '09:00'}
{'Course': 'Análise Matemática II', 'Class': 'TP2', 'Day': 'Thursday', 'Start Time': '09:00'}
{'Course': 'Tópicos de Física Moderna', 'Class': 'T1', 'Day': 'Monday', 'Start Time': '09:30'}
{'Course': 'Tópicos de Física Moderna', 'Class': 'T1', 'Day': 'Friday', 'Start Time': '09:30'}
{'Course': 'Princípios de Programação Procedimental', 'Class': 'TP8', 'Day': 'Monday', 'Start Time': '11:00'}
{'Course': 'Princípios de Programação Procedimental', 'Class': 'TP9', 'Day': 'Monday', 'Start Time': '11:00'}
{'Course': 'Estatística', 'Class': 'TP2', 'Day': 'Tuesday', 'Start Time': '11:00'}
{'Course': 'Estatística', 'Class': 'TP2', 'Day': 'Tuesday', 'Start Time': '11:00'}
{'Course': 'Análise Matemática II', 'Class': 'TP4', 'Day': 'Tuesday', 'Start Time': '11:00'}
{'Course': 'Análise Matemática II', 'Class': 'TP4', 'Day': 'Thursday', 'Start Time': '11:00'}
{'Course': 'Princípios de Programação Procediment

In [84]:
# helper function to export timetables
def save_timetable(timetable, filename='a_timetable.json',fitness_value="not calculated"):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(sorted(timetable, key=lambda x: x['Course']) + [{'fitness' : fitness_value}], f, indent=4, ensure_ascii=False)


# Fitness function
Evaluates the quality of the timetable encoded in each solution. Solutions that do not take into account the restrictions of the problem should be penalized. Lower values are better.

In [85]:
def fitness(chromosome):
    timetable = chromosome_to_timetable(chromosome)
    penalty = 0
    
    # Penalize if the number of selected classes is not 11
    num_selected_classes = sum(chromosome)
    penalty += abs(num_selected_classes - 11) * 100  # Heavy penalty for not having exactly 11 classes
    
    # Check for overlapping classes
    time_slots = {}
    for class_info in timetable:
        # Create a key for the time slot (day, start time)
        key = (class_info["Day"], class_info["Start Time"])
        
        # Check if the key is already in the dictionary (overlapping class)
        if key in time_slots:
            penalty += 100  # Penalize overlapping classes
        else:
            time_slots[key] = True  # Add the time slot to the dictionary
    
    # Check class distribution
    class_counts = {}
    for class_info in timetable:
        course = class_info["Course"]
        class_type = class_info["Class"]
        
        # Extract the general class type (e.g., TP1 -> TP, T1 -> T1, PL1 -> PL)
        general_type = ''.join(filter(str.isalpha, class_type))  # Extract only letters
        
        if course not in class_counts:
            class_counts[course] = {}
        if general_type not in class_counts[course]:
            class_counts[course][general_type] = 0
        class_counts[course][general_type] += 1
    
    # Check against required distribution
    required_distribution = {
        "Tópicos de Física Moderna": {"T": 2, "TP": 1},
        "Princípios de Programação Procedimental": {"TP": 2},
        "Comunicação Técnica": {"T": 1, "PL": 1},
        "Estatística": {"TP": 2},
        "Análise Matemática II": {"TP": 2}
    }
    
    for course, required in required_distribution.items():
        if course not in class_counts:
            # Penalize missing courses
            penalty += sum(required.values()) * 100
        else:
            for class_type, count in required.items():
                if class_type not in class_counts[course]:
                    # Penalize missing class types
                    penalty += count * 100
                else:
                    # Penalize incorrect counts
                    penalty += abs(class_counts[course][class_type] - count) * 100
    
    return penalty

# Crossover
You should program a crossover operator.

In [99]:
def crossover(parent1, parent2):
    # TODO
    # Select crossover points
    crossover_point1 = random.randint(0, len(parent1) - 1)
    crossover_point2 = random.randint(crossover_point1, len(parent1) - 1)

    # VARIATION - SINGLE POINT CROSSOVER
    # child = parent1[:crossover_point1] + parent2[crossover_point1:]
    
    # VARIATION - TWO POINT CROSSOVER
    # Create child by combining parts of parent1 and parent2
    child = parent1[:crossover_point1] + parent2[crossover_point1:crossover_point2] + parent1[crossover_point2:]

    # VARIANTION - UNIFORM CROSSOVER
    # Create child by selecting genes from parent1 and parent2 randomly
    # child = [parent1[i] if random.random() < 0.5 else parent2[i] for i in range(len(parent1))]
    
    
    return child



# Mutation operator
You should program a crossover operator.

In [96]:
def mutate(chromosome, mutation_rate):
    # TODO
    # Flip Mutation with mutation_rate probability (can be multiple flip mutation)
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            # Flip the bit
            chromosome[i] = 1 - chromosome[i]
    
    return chromosome



# Parent Selection
You should program the tournament selection mechanism

In [88]:
def tournament_selection(population, k=5):
    # TODO YOUR CODE HERE
    # Note: use sorted(<...>, key=fitness) to sort by value of the fitness function call
    selected = random.sample(population, k)
    selected = sorted(selected, key=fitness)
    # Check all the fitness values of the selected chromosomes
    #print([fitness(chromosome) for chromosome in selected])

    return selected[0], selected[1] # Return the two best chromosomes aka the ones with the lowest fitness value



In [108]:
def genetic_algorithm(pop_size=100, generations=1000, mutation_rate=0.02): # default values. We can change them to see if we have better results
    population = [generate_chromosome() for _ in range(pop_size)]
    for gen in range(generations):
        population = sorted(population, key=fitness)
        if fitness(population[0]) == 0:
            break
        new_population = [population[0]]
        while len(new_population) < pop_size:
            p1, p2 = tournament_selection(population)
            child = crossover(p1, p2)   # cuidado com as alterações usem np.copy ou equivalentes
            child = mutate(child, mutation_rate)
            new_population.append(child)
        population = new_population
        print(f"Generation {gen + 1}, Best Fitness: {fitness(population[0])}")
    return population[0]

for i in range(5):
    random.seed(i)
    best_timetable = genetic_algorithm()
    print ("---------------------------------------")

    # best_timetable = chromosome_to_timetable(best_timetable)
    # for i in best_timetable:
    #     print(i)

# Notas: Se aumentarmos o mutation_rate, o algoritmo converge mais lentamente, mas se o diminuirmos, o algoritmo converge mais rapidamente.
# Atenção: Diminiuir até um certo ponto. rate=0.001 já não é bom. O melhor é 0.02


Generation 1, Best Fitness: 2200
Generation 2, Best Fitness: 1500
Generation 3, Best Fitness: 1200
Generation 4, Best Fitness: 900
Generation 5, Best Fitness: 700
Generation 6, Best Fitness: 600
Generation 7, Best Fitness: 500
Generation 8, Best Fitness: 500
Generation 9, Best Fitness: 400
Generation 10, Best Fitness: 300
Generation 11, Best Fitness: 100
Generation 12, Best Fitness: 100
Generation 13, Best Fitness: 100
Generation 14, Best Fitness: 100
Generation 15, Best Fitness: 100
Generation 16, Best Fitness: 100
---------------------------------------
Generation 1, Best Fitness: 2000
Generation 2, Best Fitness: 1200
Generation 3, Best Fitness: 900
Generation 4, Best Fitness: 700
Generation 5, Best Fitness: 500
Generation 6, Best Fitness: 300
Generation 7, Best Fitness: 300
Generation 8, Best Fitness: 100
Generation 9, Best Fitness: 100
---------------------------------------
Generation 1, Best Fitness: 1300
Generation 2, Best Fitness: 1300
Generation 3, Best Fitness: 1200
Generatio